In [1]:
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import Dataset


In [45]:
#train = np.load(r"C:\Users\erics\Documents\Programme\Bachelorarbeit\train_data100kWithLabels.npz")
#val = np.load(r"C:\Users\erics\Documents\Programme\Bachelorarbeit\val_data100kBWithLabels.npz")
train = np.load(r"C:\Users\erics\Documents\Programme\Bachelorarbeit\train_data30kBuffer3.npz")
val = np.load(r"C:\Users\erics\Documents\Programme\Bachelorarbeit\val_data8kBuffer3.npz")
train_data = train['data']
train_labels = train['labels']
val_data = val['data']
val_labels = val['labels']
print(f"val_data {val_data.shape}")
print(f"train_data {train_data.shape}")

val_data (8000, 4, 84, 84)
train_data (30000, 4, 84, 84)


In [46]:
#train_data = train_data[:,None,:,:]
#val_data = val_data[:,None,:,:]

train_labels = train_labels.astype(dtype=np.float32)
val_labels =  val_labels.astype(dtype=np.float32)

In [47]:
print(f"val_data {val_labels.shape}")
train_labels = train_labels.reshape(-1,16)
val_labels = val_labels.reshape(-1,16)
print(f"val_data {val_labels.shape}")

val_data (8000, 4, 4)
val_data (8000, 16)


In [12]:
class Net(nn.Module):
    def __init__(self, num_channels):
        super(Net, self).__init__()
        
        self.num_channels = num_channels

        self.conv1 = nn.Conv2d(num_channels, 32, kernel_size=4, stride=2, padding=1)  # 42 x 42
        self.conv2 = nn.Conv2d(32, 32, 2, 2, 1)  # 21 x 21
        self.conv3 = nn.Conv2d(32, 64, 2, 2, 1)  # 11 x 11
        self.conv4 = nn.Conv2d(64, 64, 2, 2, 1)  # 6 x 6
        self.flat1 = nn.Flatten()
        self.dense1 = nn.Linear(3136, 256) # 6x6x 64 = 2304
        self.dense2 = nn.Linear(256, 4*num_channels)
        
        self.BN0 = nn.BatchNorm1d(256)
        self.act = nn.ReLU(inplace=True)
        
    def forward(self, x):
        
        h = self.act(self.conv1(x))
        #print("conv1: " + str(h.size()))
        h = self.act(self.conv2(h))
        #print("conv2: " + str(h.size()))
        h = self.act(self.conv3(h))
        #print("conv3: " + str(h.size()))
        h = self.act(self.conv4(h))
        #print("conv4: " + str(h.size()))
        h = self.flat1(h)
        #print(h.size())
        h = self.act(self.BN0(self.dense1(h)))
        #print(h.size())
        return self.dense2(h)

In [26]:
epochs = 80
batch_size = 32
lr = 0.0001
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [14]:
model = Net(train_data.shape[1]).to(device) #labels.shape[1] ).to(device)
print(model)

Net(
  (conv1): Conv2d(4, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (conv2): Conv2d(32, 32, kernel_size=(2, 2), stride=(2, 2), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(2, 2), stride=(2, 2), padding=(1, 1))
  (conv4): Conv2d(64, 64, kernel_size=(2, 2), stride=(2, 2), padding=(1, 1))
  (flat1): Flatten(start_dim=1, end_dim=-1)
  (dense1): Linear(in_features=3136, out_features=256, bias=True)
  (dense2): Linear(in_features=256, out_features=16, bias=True)
  (BN0): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act): ReLU(inplace=True)
)


In [15]:
opt = optim.Adam(model.parameters(), lr=lr)
loss = nn.MSELoss()

In [23]:
def fit(model, inpX, inp_labels):
    inpX = inpX#.to(device)
    inp_labels = inp_labels#.to(device)
    model.train
    train_loss = []
    correct = 0
    
    for i in range(0, len(inp_labels)-batch_size, batch_size):
        if i % 10000 == 0:
            print(f"i: {i/10000}")   
        
        running_loss = 0.0
        X = inpX[i:(i+batch_size)].to(device)
        labels = inp_labels[i:i+batch_size].to(device)
        #print(X.device)
        
        opt.zero_grad()

        inp = model(X)#.detach().cpu()
        #inp = inp.view(inp.shape[0])
        output = loss(inp, labels)
        
        correct += (torch.round(inp) == labels).float().sum()

        
        
        output.retain_grad()
        output.backward()
        running_loss += output.item()
        

        opt.step()

        train_loss.append(running_loss)
    accuracy = correct/len(inp_labels)
    return train_loss, accuracy

In [24]:
def validate(model, X, labels):
    inpX = X#.to(device)
    inp_labels = labels#.to(device)
    model.eval()
    val_loss = []
    correct = 0  
    
    with torch.no_grad():
        for i in range(0, len(inp_labels)-batch_size, batch_size):
            if i % 10000 == 0:
                print(f"i: {i/10000}")   

            running_loss = 0.0
            X = inpX[i:(i+batch_size)].to(device)
            labels = inp_labels[i:i+batch_size].to(device)
            #print(X.device)

            inp = model(X)#.detach().cpu()
            #inp = inp.view(inp.shape[0])

            correct += (torch.round(inp) == labels).float().sum()


            output = loss(inp, labels)
            running_loss += output.item()
            
            val_loss.append(running_loss)
            
    accuracy = correct/len(inp_labels)
    return val_loss, accuracy

In [48]:
train_loss = []
val_loss = []
train_accuracies = []
val_accuracies = []
torch.backends.cudnn.benchmark = True #choose best kernel for computation

for epoch in range(epochs):
    print(f"Epoch {epoch+1} of {epochs}")

    train_epoch_loss, train_accuracy = fit(model, torch.from_numpy(train_data), torch.from_numpy(train_labels))
    torch.cuda.empty_cache()
    val_epoch_loss, val_accuracy = validate(model, torch.from_numpy(val_data), torch.from_numpy(val_labels))
    print(f"Train Loss: {np.mean(np.asarray(train_epoch_loss))}, Train Accuracy: {train_accuracy}")
    print(f"Val Loss: {np.mean(np.asarray(val_epoch_loss))}, Validation Accuracy: {val_accuracy}")
    train_loss.append(np.mean(np.asarray(train_epoch_loss)))
    val_loss.append(np.mean(np.asarray(val_epoch_loss)))
    train_accuracies.append(train_accuracy)
    val_accuracies.append(val_accuracy)
    
    
print(f"Train Loss: {train_loss}, Train Accuracy: {train_accuracies}")
print(f"Val Loss: {val_loss}, Validation Accuracy: {val_accuracies}")

Epoch 1 of 80
i: 0.0
i: 2.0
i: 0.0
Train Loss: 0.06450754314812104, Train Accuracy: 15.19123363494873
Val Loss: 0.05501704945801252, Validation Accuracy: 15.254000663757324
Epoch 2 of 80
i: 0.0
i: 2.0
i: 0.0
Train Loss: 0.049677193568365426, Train Accuracy: 15.357033729553223
Val Loss: 0.05318641004313427, Validation Accuracy: 15.273250579833984
Epoch 3 of 80
i: 0.0
i: 2.0
i: 0.0
Train Loss: 0.046474179495503705, Train Accuracy: 15.389900207519531
Val Loss: 0.05049479993651191, Validation Accuracy: 15.303125381469727
Epoch 4 of 80
i: 0.0
i: 2.0
i: 0.0
Train Loss: 0.04474106737936319, Train Accuracy: 15.409000396728516
Val Loss: 0.0506600650797886, Validation Accuracy: 15.303376197814941
Epoch 5 of 80
i: 0.0
i: 2.0
i: 0.0
Train Loss: 0.043251980603011594, Train Accuracy: 15.42473316192627
Val Loss: 0.049648710807044825, Validation Accuracy: 15.30612564086914
Epoch 6 of 80
i: 0.0
i: 2.0
i: 0.0
Train Loss: 0.042177500537215966, Train Accuracy: 15.4353666305542
Val Loss: 0.0495186064272640

i: 2.0
i: 0.0
Train Loss: 0.03220372571507538, Train Accuracy: 15.594066619873047
Val Loss: 0.04623421429690108, Validation Accuracy: 15.375125885009766
Epoch 49 of 80
i: 0.0
i: 2.0
i: 0.0
Train Loss: 0.031603100717568666, Train Accuracy: 15.595200538635254
Val Loss: 0.0427121719622229, Validation Accuracy: 15.400500297546387
Epoch 50 of 80
i: 0.0
i: 2.0
i: 0.0
Train Loss: 0.029109021362509646, Train Accuracy: 15.611599922180176
Val Loss: 0.04257099174082758, Validation Accuracy: 15.400500297546387
Epoch 51 of 80
i: 0.0
i: 2.0
i: 0.0
Train Loss: 0.027538593013630732, Train Accuracy: 15.616033554077148
Val Loss: 0.040706139929353236, Validation Accuracy: 15.414000511169434
Epoch 52 of 80
i: 0.0
i: 2.0
i: 0.0
Train Loss: 0.02613135694407857, Train Accuracy: 15.622233390808105
Val Loss: 0.04141635143284099, Validation Accuracy: 15.407751083374023
Epoch 53 of 80
i: 0.0
i: 2.0
i: 0.0
Train Loss: 0.02673022893208736, Train Accuracy: 15.620599746704102
Val Loss: 0.04158669986937898, Validatio

In [49]:
inp = model(torch.from_numpy(val_data)[20].to(device)[None,:,:,:])
print(inp)

tensor([[72.0044, 54.9709, 30.0603, 36.9084, 70.0408, 56.8941, 31.9836, 38.9517,
         68.0544, 59.0159, 33.9808, 40.9164, 66.0493, 60.9065, 35.8999, 42.9465]],
       device='cuda:0', grad_fn=<AddmmBackward0>)


In [50]:
print(val_labels[20])

[72. 55. 30. 37. 70. 57. 32. 39. 68. 59. 34. 41. 66. 61. 36. 43.]


In [51]:
torch.save(model.state_dict(), "GT_from_Images_Buffer_400E_3-5")